In [7]:
import os
import numpy as np
import librosa
import soundfile as sf
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score

# Step 1: Feature Extraction Function
def extract_features(file_path):
    """Extract audio features from a WAV file (only MFCC)."""
    try:
        audio, sr = librosa.load(file_path, sr=22050)
        # Extract MFCC features
        mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13).mean(axis=1)
        return mfcc
    except Exception as e:
        print(f"Error extracting features from {file_path}: {e}")
        return None

# Step 2: Prepare the Dataset
def load_dataset(data_dir):
    """Load the dataset and extract features and labels."""
    features = []
    labels = []
    
    for label in os.listdir(data_dir):
        label_dir = os.path.join(data_dir, label)
        if os.path.isdir(label_dir):
            for file in os.listdir(label_dir):
                file_path = os.path.join(label_dir, file)
                if file_path.endswith('.wav'):
                    mfcc = extract_features(file_path)
                    if mfcc is not None:
                        features.append(mfcc)
                        labels.append(label)

    return np.array(features), np.array(labels)

# Step 3: Load Data
data_directory = r'C:\\Users\\admin\\Documents\\stress_analysis\\emotional_state\\TESS Toronto emotional speech set data'  # Update this with the correct path
X, y = load_dataset(data_directory)

# Step 4: Encode Labels
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

# Step 5: Split the Dataset
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Step 6: Train the Random Forest Classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Step 7: Predict and Evaluate
y_pred = clf.predict(X_test)
print(f"Model Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")
print(classification_report(y_test, y_pred, target_names=encoder.classes_))

# Step 8: Save the Model and Encoder
joblib.dump(clf, "emotion_detection_model.joblib")
joblib.dump(encoder, "label_encoder.joblib")

# Load the trained model for prediction
model = joblib.load("emotion_detection_model.joblib")

# Step 9: Audio File Prediction
def predict_emotion(file_path):
    """Predict the emotion of the given audio file."""
    features = extract_features(file_path)
    if features is not None:
        expected_feature_count = model.n_features_in_
        print(f"Model expects {expected_feature_count} features.")
        
        if features.shape[0] != expected_feature_count:
            print(f"Warning: Expected {expected_feature_count} features, but got {features.shape[0]}.")
            return
        
        features = features.reshape(1, -1)  # Reshape to match model input
        prediction = model.predict(features)
        emotion = encoder.inverse_transform(prediction)
        print(f"Predicted Emotion: {emotion[0]}")
    else:
        print(f"Failed to extract features from {file_path}")

# Example usage: Predict from a pre-recorded audio file
pre_recorded_file = r'C:\\Users\\admin\\Documents\\stress_analysis\\emotional_state\\TESS Toronto emotional speech set data\\OAF_fear\\OAF_base_fear.wav'
predict_emotion(pre_recorded_file)


Model Accuracy: 93.75%
                        precision    recall  f1-score   support

              OAF_Fear       0.90      0.88      0.89        52
 OAF_Pleasant_surprise       0.98      0.89      0.93        47
               OAF_Sad       0.98      0.92      0.95        49
             OAF_angry       1.00      1.00      1.00        37
           OAF_disgust       0.97      0.92      0.95        39
             OAF_happy       0.81      0.95      0.88        37
           OAF_neutral       0.83      0.97      0.89        30
             YAF_angry       0.94      0.86      0.90        37
           YAF_disgust       0.95      0.92      0.94        39
              YAF_fear       0.89      0.92      0.91        37
             YAF_happy       0.95      0.97      0.96        40
           YAF_neutral       0.98      0.95      0.96        43
YAF_pleasant_surprised       0.95      1.00      0.98        40
               YAF_sad       1.00      1.00      1.00        33

              a